# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_csv = "output_data/cities.csv"
weather_data = pd.read_csv(weather_data_csv)
del weather_data["Unnamed: 0"]
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lebu,-37.62,-73.65,285.35,80,89,0.43,CL,1589431718
1,Hobart,-42.88,147.33,289.15,51,20,6.20,AU,1589431718
2,Rikitea,-23.12,-134.97,298.60,84,0,4.20,PF,1589431643
3,Hilo,19.73,-155.09,300.15,61,1,4.10,US,1589431719
4,Redmond,47.67,-122.12,287.04,67,79,0.87,US,1589431682


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
weather_data.dropna(inplace = True) 
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_data["Humidity"]),
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lebu,-37.62,-73.65,285.35,80,89,0.43,CL,1589431718
1,Hobart,-42.88,147.33,289.15,51,20,6.20,AU,1589431718
2,Rikitea,-23.12,-134.97,298.60,84,0,4.20,PF,1589431643
3,Hilo,19.73,-155.09,300.15,61,1,4.10,US,1589431719
4,Redmond,47.67,-122.12,287.04,67,79,0.87,US,1589431682


In [6]:
weather_data.dropna(inplace = True) 
new_data = weather_data.loc[(weather_data['Lat'] > 0) & (weather_data['Humidity'] < 50) & (weather_data['Cloudiness'] == 0)]

new_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,Mikhaylovskoye,51.82,79.72,302.39,25,0,1.81,RU,1589431735
72,Dārāb,28.75,54.54,299.54,15,0,1.56,IR,1589431752
108,Aksu,41.12,80.26,302.02,16,0,3.76,CN,1589431770
121,Atar,20.52,-13.05,296.28,25,0,4.48,MR,1589431777
126,Barwāni,22.03,74.90,311.48,24,0,3.30,IN,1589431779


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = new_data

In [9]:
params = {
    "radius": 5000, 
    "types": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
     # print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    
    try:
        print(f"Closest hotel is {response['results'][0]['name']}.") 
        hotel_df.loc[index, 'Hotel Name'] = results[0]['vicinity']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Closest hotel is Terem".
Closest hotel is Naghshe shapoor Hotel.
Closest hotel is Pudong Holiday Hotel.
Closest hotel is Oumou Elghoura.
Closest hotel is Mahalaxmi Guest House.
Missing field/result... skipping.
Missing field/result... skipping.
Closest hotel is Orient Apartments.
Missing field/result... skipping.
Closest hotel is Bungalows Guesthouse.
Closest hotel is ООО "Гостиничный комплекс "Петропавловск".
Closest hotel is Sai Deluxe Shahapur.
Closest hotel is Ахмадын амралт сувилал.
Closest hotel is Hotel Haveli.
Missing field/result... skipping.
Closest hotel is Novotel Athenes.
Closest hotel is Hotel Santa Fe.
Closest hotel is Kemping "Malinovoye Ozero".
Closest hotel is Mosterøy.
Closest hotel is Sur Plaza Hotel.
Closest hotel is Reyhani Kasri Hotel.
Closest hotel is Heuan Na Khee.
Closest hotel is Hilton Garden Inn Albuquerque North/Rio Rancho.


In [10]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
34,Mikhaylovskoye,51.82,79.72,302.39,25,0,1.81,RU,1589431735,"Ул. Зеленая, 13, Mikhaylovskoye"
72,Dārāb,28.75,54.54,299.54,15,0,1.56,IR,1589431752,"Darab, Daneshjoo Boulevard, Pasdaran Blv."
108,Aksu,41.12,80.26,302.02,16,0,3.76,CN,1589431770,"交通西路1号, Jiaotong Road, Aksu City, Aksu Prefecture"
121,Atar,20.52,-13.05,296.28,25,0,4.48,MR,1589431777,"Mbarka Oumara Derriere l'hopital regional, أطار‎"
126,Barwāni,22.03,74.90,311.48,24,0,3.30,IN,1589431779,Barwani


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))